In [1]:
# set gpu
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import sys
from pathlib import Path
base_dir = str(Path(sys.path[0]) / "..")
sys.path.insert(0, base_dir)
print(sys.path)

import pandas as pd
import random
from evaluation.sentiment import classify_sentiment_with_vader, parse_vader_result, classify_sentiment
from evaluation.sentence_processing import count_questions

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

['/home/user/kew/projects/understanding-ctx-aug/analysis/..', '/home/user/kew/projects/understanding-ctx-aug/analysis', '/home/user/kew/anaconda3/envs/unsup_ctrl/lib/python38.zip', '/home/user/kew/anaconda3/envs/unsup_ctrl/lib/python3.8', '/home/user/kew/anaconda3/envs/unsup_ctrl/lib/python3.8/lib-dynload', '', '/home/user/kew/anaconda3/envs/unsup_ctrl/lib/python3.8/site-packages', '/home/user/kew/projects/understanding-ctx-aug/src/transformers/src']


/home/user/kew/anaconda3/envs/unsup_ctrl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
data = {}
data_dir = '../resources/models/seed_23/KGD/bart_mini-SI_bart/'
for checkpoint_dir in sorted(Path(data_dir).glob('checkpoint-*')):
    if (checkpoint_dir / 'outputs').exists():
        # get the output file
        # output_file = checkpoint_dir / 'outputs' / 'generations_test_freq_seed=42_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9.txt'
        output_file = checkpoint_dir / 'outputs' / 'generations_test_freq_seed=983_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9_ctxt=5-train_questions-10.txt'
        if not output_file.exists():
            # print(f'File {output_file} does not exist')
            continue

        # generations_test_freq_seed=0_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9_ctxt=5-pos_sents-5.txt
        # generations_test_freq_seed=0_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9_ctxt=5-train_questions-10.txt
        # generations_test_freq_seed=0_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9.txt
        # 'generations_test_freq_seed=983_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9_ctxt=5-pos_sents-5.txt'
        # 'generations_test_freq_seed=983_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9_ctxt=5-train_questions-10.txt'
        # 'generations_test_freq_seed=983_ml=40_lp=1.0_ns=1_bs=4_ds=1_temp=0.7_tk=0_tp=0.9.txt'

        # sample 20 generations from the output file
        with open(output_file, 'r', encoding='utf8') as f:
            generations = random.sample([line.strip() for line in f], 100)

            data[checkpoint_dir.name] = generations

# sort the keys
data = dict(sorted(data.items(), key=lambda x: int(x[0].split('-')[1])))
print(len(data))
print(data.keys())

12
dict_keys(['checkpoint-2420', 'checkpoint-4840', 'checkpoint-7260', 'checkpoint-14520', 'checkpoint-21780', 'checkpoint-29040', 'checkpoint-36300', 'checkpoint-43560', 'checkpoint-50820', 'checkpoint-58080', 'checkpoint-65340', 'checkpoint-72600'])


In [14]:
# classify the sentiment of the generations
# for key, generations in data.items():
#     sents = classify_sentiment(generations, 'distilbert-base-uncased-finetuned-sst-2-english', 20)
#     data[key] = {'generations': generations, 'label': [i['label'] for i in sents]}

for key, generations in data.items():
    sents = count_questions(generations)
    data[key] = {'generations': generations, 'label': sents}
    
# combine results into dataframe
dfs = []
for checkpoint, checkpoint_data in data.items():
    df = pd.DataFrame(checkpoint_data)
    dfs.append(df)

df = pd.concat(dfs, keys=data.keys())
df.index.names = ['Checkpoint', None]

df



generations  label
Checkpoint                                                                   
checkpoint-2420  0               I don't know that. Did you know that!      0
                 1   I wonder how they can be picked up and heard b...      0
                 2                         I think it is a good point.      0
                 3                   I wonder if it is a lot of money.      0
                 4                    I think that is a lot of people.      0
...                                                                ...    ...
checkpoint-72600 95  I agree. I don't know that the NFL has no writ...      0
                 96  Yeah, I wonder if they were the same restauran...      0
                 97  I wonder if it was a lot of money. I wonder if...      0
                 98  I don't know that. Did you know that ESPN won ...      0
                 99  Yeah, I don't know that. Did you know that the...      1

[1200 rows x 2 columns]

In [15]:
for i, row in df.loc['checkpoint-2420'].iterrows():
    print(row['label'], row['generations'])

0 I don't know that. Did you know that!
0 I wonder how they can be picked up and heard by an AM.
0 I think it is a good point.
0 I wonder if it is a lot of money.
0 I think that is a lot of people.
1 I wonder if they are the top three wealthiest presidents?
1 I don't know that. Did you know that?
0 I don't know that the highest score
0 I wonder if it is a razor to cut out the moon.
0 I think that is the highest score
1 I don't know that. I don't think that. Did you know that?
0 I think that is amazing. I wonder if they are a lot of money.
1 I don't know that. Do you know that I do not know that?
1 I don't know that. Did you know that?
0 I think it is a networks that consists of private, public.
0 I don't know that. Did you know that it is a lot of money.
0 I'm not sure. I don't know that it's a lot of fantasy!
0 I wonder if it is a lot of the same time.
0 I don't know that. Did you know that is almost no correlation between cell phone.
0 I don't know that. Did you know that is the same

In [16]:
# inspect generations and sentiments for a specific checkpoint
checkpoint = 'checkpoint-14520'
for i, row in df.loc[checkpoint].iterrows():
    print(row['label'], row['generations'])


1 I don't know that. Did you know that NFL Commissioner Roger Goodell attended the game?
0 I did not know that. Hugh Laurie is the highest paid actor ever.
0 I don't know that. I don't think that it was not uncommon.
1 I didn't know that. Do you know that Baseball is so popular?
0 I don't know that. I'm not sure. I wonder if they are.
1 I don't know that. Did you know that the Sports commonly called football?
1 Yes, I didn't know that. Did you know there are over 750 episode?
1 I think that is a good idea. Did you know the first president of Zimbabwe?
1 I think that is a lot of money. Do you know that?
1 That's crazy. Did you know that?
1 I think that is a good idea. Did you know the Average Lifetime Earnings of an Engineer?
1 I don't know that. Did you know that the University of Iowa's locker room for visiting football teams?
1 I did not know that. Do you know that the highest score?
0 I have no idea. I don't know that New York's area code is 212.
1 I think that is a lot of money. Di

In [17]:
for i, row in df.loc['checkpoint-72600'].iterrows():
    print(row['label'], row['generations'])


0 yeah, I didn't know that.
1 Yeah, I don't know that. Do you know the NFL uses tracking chips embedded?
0 I wonder if the National Football League is a professional American football league.
1 I don't know that. Did you know that Baseball is so popular in Japan?
1 I don't know that. Did you know that the author of A Games of Thrones?
1 I'm not sure. Did you know the University of Iowa's locker room for visiting football teams?
1 I wonder if it is a lot of money. Did you know that the average NFL game only has 11 minutes of live gameplay?
0 I did not know that. I didn't know that the University of Iowa's locker room.
1 I'm not sure. Did you know that there is a lot of money?
1 That's pretty cool. Did you know that Former Partiots RB Benjarvus Green-Ellis?
0 I don't know that. I don't think that would be interesting. I wonder if they do.
0 Yes they are the highest score in a football game.
1 I think it is a lot of money. Did you know he's shown the ability to rebound from adversity?
0 I